<a href="https://colab.research.google.com/github/Tessellate-Imaging/Monk_Object_Detection/blob/master/application_model_zoo/Example%20-%20Protective%20gear%20-%20helmet%20and%20vest%20detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of contents


## 1. Installation Instructions



## 2. Use trained model to detect safety vest and helmet



## 3. How to train using MMdetection wrapper and Personal-Protective-Equipment dataset

# Installation

 - Run these commands
     
     - git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git
     
     - cd Monk_Object_Detection/16_mmdet/installation
     
 - Select the right file and run
 
     - chmod +x install.sh && ./install.sh

In [ ]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

In [ ]:
! cd Monk_Object_Detection/16_mmdet/installation && chmod +x install.sh && ./install.sh

# Use already trained model for demo

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/16_mmdet/lib")

In [ ]:
from infer_engine import Infer

In [ ]:
gtf = Infer();

In [ ]:
# Download trained model

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1P8dUOhXTeMu4NmYnzJA-xVbz51sCIkRk' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1P8dUOhXTeMu4NmYnzJA-xVbz51sCIkRk" -O obj_ppe_trained.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq obj_ppe_trained.zip

In [ ]:
gtf.Model_Params("work_dirs/config_updated/config_updated.py", 
                 "work_dirs/config_updated/latest.pth")

In [ ]:
import cv2
def print_result(input_img_name, result, output_img_name, thresh=0.3):
    img = cv2.imread(input_img_name, 1);
    for i in range(len(result)):
        for j in range(len(result[i])):
            x1 = int(result[i][j][0]);
            y1 = int(result[i][j][1]);
            x2 = int(result[i][j][2]);
            y2 = int(result[i][j][3]);
            score = float(result[i][j][4]);
            if(score > thresh):
                if(i == 0):
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,255), 4)
                elif(i == 1):
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 4)
                elif(i == 2):
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0,0,255), 4)
    cv2.imwrite(output_img_name, img);

In [ ]:
result = gtf.Predict(img_path="work_dirs/test/7.jpg",
           out_img_path="result.jpg",
           thresh=0.1);

print_result("work_dirs/test/7.jpg", result, "7.jpg", thresh=0.5)

In [ ]:
result = gtf.Predict(img_path="work_dirs/test/6.jpg",
           out_img_path="result.jpg",
           thresh=0.1);

print_result("work_dirs/test/6.jpg", result, "6.jpg", thresh=0.5)

In [ ]:
result = gtf.Predict(img_path="work_dirs/test/5.jpg",
           out_img_path="result.jpg",
           thresh=0.1);

print_result("work_dirs/test/5.jpg", result, "5.jpg", thresh=0.9)

In [ ]:
result = gtf.Predict(img_path="work_dirs/test/2.jpg",
           out_img_path="result.jpg",
           thresh=0.1);

print_result("work_dirs/test/2.jpg", result, "2.jpg", thresh=0.3)

In [ ]:
result = gtf.Predict(img_path="work_dirs/test/3.jpg",
           out_img_path="result.jpg",
           thresh=0.1);

print_result("work_dirs/test/3.jpg", result, "3.jpg", thresh=0.3)

In [ ]:
result = gtf.Predict(img_path="work_dirs/test/4.jpg",
           out_img_path="result.jpg",
           thresh=0.1);

print_result("work_dirs/test/4.jpg", result, "4.jpg", thresh=0.95)

In [ ]:
result = gtf.Predict(img_path="work_dirs/test/1.jpg",
           out_img_path="result.jpg",
           thresh=0.1);

print_result("work_dirs/test/1.jpg", result, "1.jpg", thresh=0.3)

# Training your own detector

## Dataset
 - Credits: https://github.com/ciber-lab/pictor-ppe

### Unzip the file downloaded from gdrive in the mentioned directory structure

    pictor-ppe
       |----------Images
       |----------Labels


In [ ]:
! mkdir dataset

! mkdir dataset/train
! mkdir dataset/train/images
! mkdir dataset/valid
! mkdir dataset/valid/images
! mkdir dataset/test
! mkdir dataset/test/images

## Data conversion - Current format to Monk format

In [ ]:
import os
import sys
import cv2
import numpy as np
import pandas as pd

In [ ]:
f = open("pictor-ppe/Labels/pictor_ppe_crowdsourced_approach-01_train.txt");
lines = f.readlines();
f.close();

In [ ]:
from tqdm import tqdm
combined = [];

for i in tqdm(range(len(lines))):
    stats = lines[i][:len(lines[i])-1].split("\t");
    img_name = stats[0];
    img = cv2.imread("pictor-ppe/Images/" + img_name)
    wr = "";
    for j in range(1, len(stats)):
        tmp = stats[j].split(",");
        x1 = tmp[0];
        y1 = tmp[1];
        x2 = tmp[2];
        y2 = tmp[3];
        label = tmp[4];
        if(label == "0"):
            label = "helmet";
        elif(label == "1"):
            label = "safety_vest";
        else:
            label = "person";
        wr += x1 + " " + y1 + " " + x2 + " " + y2 + " " + label + " ";
        
    wr = wr[:len(wr)-1]
    combined.append([str(i) + ".jpg", wr]);
        
    cv2.imwrite("dataset/train/images/" + str(i) + ".jpg", img)


In [ ]:
df = pd.DataFrame(combined, columns = ['Img_Name', 'Label']);
df.to_csv("dataset/train/train_labels.csv", index=False);

In [ ]:
f = open("pictor-ppe/Labels/pictor_ppe_crowdsourced_approach-01_valid.txt");
lines = f.readlines();
f.close();

In [ ]:
from tqdm import tqdm
combined = [];

for i in tqdm(range(len(lines))):
    stats = lines[i][:len(lines[i])-1].split("\t");
    img_name = stats[0];
    img = cv2.imread("pictor-ppe/Images/" + img_name)
    wr = "";
    for j in range(1, len(stats)):
        tmp = stats[j].split(",");
        x1 = tmp[0];
        y1 = tmp[1];
        x2 = tmp[2];
        y2 = tmp[3];
        label = tmp[4];
        if(label == "0"):
            label = "helmet";
        elif(label == "1"):
            label = "safety_vest";
        else:
            label = "person";
        wr += x1 + " " + y1 + " " + x2 + " " + y2 + " " + label + " ";
        
    wr = wr[:len(wr)-1]
    combined.append([str(i) + ".jpg", wr]);
        
    cv2.imwrite("dataset/valid/images/" + str(i) + ".jpg", img)

In [ ]:
df = pd.DataFrame(combined, columns = ['Img_Name', 'Label']);
df.to_csv("dataset/valid/valid_labels.csv", index=False);

In [ ]:
f = open("pictor-ppe/Labels/pictor_ppe_crowdsourced_approach-01_test.txt");
lines = f.readlines();
f.close();

In [ ]:
from tqdm import tqdm
combined = [];

for i in tqdm(range(len(lines))):
    stats = lines[i][:len(lines[i])-1].split("\t");
    img_name = stats[0];
    img = cv2.imread("pictor-ppe/Images/" + img_name)
    wr = "";
    for j in range(1, len(stats)):
        tmp = stats[j].split(",");
        x1 = tmp[0];
        y1 = tmp[1];
        x2 = tmp[2];
        y2 = tmp[3];
        label = tmp[4];
        if(label == "0"):
            label = "helmet";
        elif(label == "1"):
            label = "safety_vest";
        else:
            label = "person";
        wr += x1 + " " + y1 + " " + x2 + " " + y2 + " " + label + " ";
        
    wr = wr[:len(wr)-1]
    combined.append([str(i) + ".jpg", wr]);
        
    cv2.imwrite("dataset/test/images/" + str(i) + ".jpg", img)

In [ ]:
df = pd.DataFrame(combined, columns = ['Img_Name', 'Label']);
df.to_csv("dataset/test/test_labels.csv", index=False);

## Data conversion - Monk format to COCO format

In [ ]:
import os
import numpy as np 
import cv2
import dicttoxml
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString
from tqdm import tqdm
import shutil
import json
import pandas as pd


root = "dataset/train";
img_dir = "images/";
anno_file = "train_labels.csv";

dataset_path = root;
images_folder = root + "/" + img_dir;
annotations_path = root + "/annotations/";

if not os.path.isdir(annotations_path):
    os.mkdir(annotations_path)
    
input_images_folder = images_folder;
input_annotations_path = root + "/" + anno_file;


output_dataset_path = root;
output_image_folder = input_images_folder;
output_annotation_folder = annotations_path;

tmp = img_dir.replace("/", "");
output_annotation_file = output_annotation_folder + "/instances_" + tmp + ".json";
output_classes_file = output_annotation_folder + "/classes.txt";

if not os.path.isdir(output_annotation_folder):
    os.mkdir(output_annotation_folder);

df = pd.read_csv(input_annotations_path);
columns = df.columns

delimiter = " ";


list_dict = [];
anno = [];
for i in range(len(df)):
    img_name = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(delimiter);
    for j in range(len(tmp)//5):
        label = tmp[j*5+4];
        if(label not in anno):
            anno.append(label);
    anno = sorted(anno)
    
for i in tqdm(range(len(anno))):
    tmp = {};
    tmp["supercategory"] = "master";
    tmp["id"] = i;
    tmp["name"] = anno[i];
    list_dict.append(tmp);

anno_f = open(output_classes_file, 'w');
for i in range(len(anno)):
    anno_f.write(anno[i] + "\n");
anno_f.close();



coco_data = {};
coco_data["type"] = "instances";
coco_data["images"] = [];
coco_data["annotations"] = [];
coco_data["categories"] = list_dict;
image_id = 0;
annotation_id = 0;


for i in tqdm(range(len(df))):
    img_name = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(delimiter);
    image_in_path = input_images_folder + "/" + img_name;
    img = cv2.imread(image_in_path, 1);
    h, w, c = img.shape;

    images_tmp = {};
    images_tmp["file_name"] = img_name;
    images_tmp["height"] = h;
    images_tmp["width"] = w;
    images_tmp["id"] = image_id;
    coco_data["images"].append(images_tmp);
    

    for j in range(len(tmp)//5):
        x1 = int(tmp[j*5+0]);
        y1 = int(tmp[j*5+1]);
        x2 = int(tmp[j*5+2]);
        y2 = int(tmp[j*5+3]);
        label = tmp[j*5+4];
        annotations_tmp = {};
        annotations_tmp["id"] = annotation_id;
        annotation_id += 1;
        annotations_tmp["image_id"] = image_id;
        annotations_tmp["segmentation"] = [];
        annotations_tmp["ignore"] = 0;
        annotations_tmp["area"] = (x2-x1)*(y2-y1);
        annotations_tmp["iscrowd"] = 0;
        annotations_tmp["bbox"] = [x1, y1, x2-x1, y2-y1];
        annotations_tmp["category_id"] = anno.index(label);

        coco_data["annotations"].append(annotations_tmp)
    image_id += 1;

outfile =  open(output_annotation_file, 'w');
json_str = json.dumps(coco_data, indent=4);
outfile.write(json_str);
outfile.close();

In [ ]:
import os
import numpy as np 
import cv2
import dicttoxml
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString
from tqdm import tqdm
import shutil
import json
import pandas as pd


root = "dataset/valid";
img_dir = "images/";
anno_file = "valid_labels.csv";

dataset_path = root;
images_folder = root + "/" + img_dir;
annotations_path = root + "/annotations/";

if not os.path.isdir(annotations_path):
    os.mkdir(annotations_path)
    
input_images_folder = images_folder;
input_annotations_path = root + "/" + anno_file;


output_dataset_path = root;
output_image_folder = input_images_folder;
output_annotation_folder = annotations_path;

tmp = img_dir.replace("/", "");
output_annotation_file = output_annotation_folder + "/instances_" + tmp + ".json";
output_classes_file = output_annotation_folder + "/classes.txt";

if not os.path.isdir(output_annotation_folder):
    os.mkdir(output_annotation_folder);

df = pd.read_csv(input_annotations_path);
columns = df.columns

delimiter = " ";


list_dict = [];
anno = [];
for i in range(len(df)):
    img_name = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(delimiter);
    for j in range(len(tmp)//5):
        label = tmp[j*5+4];
        if(label not in anno):
            anno.append(label);
    anno = sorted(anno)
    
for i in tqdm(range(len(anno))):
    tmp = {};
    tmp["supercategory"] = "master";
    tmp["id"] = i;
    tmp["name"] = anno[i];
    list_dict.append(tmp);

anno_f = open(output_classes_file, 'w');
for i in range(len(anno)):
    anno_f.write(anno[i] + "\n");
anno_f.close();



coco_data = {};
coco_data["type"] = "instances";
coco_data["images"] = [];
coco_data["annotations"] = [];
coco_data["categories"] = list_dict;
image_id = 0;
annotation_id = 0;


for i in tqdm(range(len(df))):
    img_name = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(delimiter);
    image_in_path = input_images_folder + "/" + img_name;
    img = cv2.imread(image_in_path, 1);
    h, w, c = img.shape;

    images_tmp = {};
    images_tmp["file_name"] = img_name;
    images_tmp["height"] = h;
    images_tmp["width"] = w;
    images_tmp["id"] = image_id;
    coco_data["images"].append(images_tmp);
    

    for j in range(len(tmp)//5):
        x1 = int(tmp[j*5+0]);
        y1 = int(tmp[j*5+1]);
        x2 = int(tmp[j*5+2]);
        y2 = int(tmp[j*5+3]);
        label = tmp[j*5+4];
        annotations_tmp = {};
        annotations_tmp["id"] = annotation_id;
        annotation_id += 1;
        annotations_tmp["image_id"] = image_id;
        annotations_tmp["segmentation"] = [];
        annotations_tmp["ignore"] = 0;
        annotations_tmp["area"] = (x2-x1)*(y2-y1);
        annotations_tmp["iscrowd"] = 0;
        annotations_tmp["bbox"] = [x1, y1, x2-x1, y2-y1];
        annotations_tmp["category_id"] = anno.index(label);

        coco_data["annotations"].append(annotations_tmp)
    image_id += 1;

outfile =  open(output_annotation_file, 'w');
json_str = json.dumps(coco_data, indent=4);
outfile.write(json_str);
outfile.close();

In [ ]:
import os
import numpy as np 
import cv2
import dicttoxml
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString
from tqdm import tqdm
import shutil
import json
import pandas as pd


root = "dataset/test";
img_dir = "images/";
anno_file = "test_labels.csv";

dataset_path = root;
images_folder = root + "/" + img_dir;
annotations_path = root + "/annotations/";

if not os.path.isdir(annotations_path):
    os.mkdir(annotations_path)
    
input_images_folder = images_folder;
input_annotations_path = root + "/" + anno_file;


output_dataset_path = root;
output_image_folder = input_images_folder;
output_annotation_folder = annotations_path;

tmp = img_dir.replace("/", "");
output_annotation_file = output_annotation_folder + "/instances_" + tmp + ".json";
output_classes_file = output_annotation_folder + "/classes.txt";

if not os.path.isdir(output_annotation_folder):
    os.mkdir(output_annotation_folder);

df = pd.read_csv(input_annotations_path);
columns = df.columns

delimiter = " ";


list_dict = [];
anno = [];
for i in range(len(df)):
    img_name = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(delimiter);
    for j in range(len(tmp)//5):
        label = tmp[j*5+4];
        if(label not in anno):
            anno.append(label);
    anno = sorted(anno)
    
for i in tqdm(range(len(anno))):
    tmp = {};
    tmp["supercategory"] = "master";
    tmp["id"] = i;
    tmp["name"] = anno[i];
    list_dict.append(tmp);

anno_f = open(output_classes_file, 'w');
for i in range(len(anno)):
    anno_f.write(anno[i] + "\n");
anno_f.close();



coco_data = {};
coco_data["type"] = "instances";
coco_data["images"] = [];
coco_data["annotations"] = [];
coco_data["categories"] = list_dict;
image_id = 0;
annotation_id = 0;


for i in tqdm(range(len(df))):
    img_name = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(delimiter);
    image_in_path = input_images_folder + "/" + img_name;
    img = cv2.imread(image_in_path, 1);
    h, w, c = img.shape;

    images_tmp = {};
    images_tmp["file_name"] = img_name;
    images_tmp["height"] = h;
    images_tmp["width"] = w;
    images_tmp["id"] = image_id;
    coco_data["images"].append(images_tmp);
    

    for j in range(len(tmp)//5):
        x1 = int(tmp[j*5+0]);
        y1 = int(tmp[j*5+1]);
        x2 = int(tmp[j*5+2]);
        y2 = int(tmp[j*5+3]);
        label = tmp[j*5+4];
        annotations_tmp = {};
        annotations_tmp["id"] = annotation_id;
        annotation_id += 1;
        annotations_tmp["image_id"] = image_id;
        annotations_tmp["segmentation"] = [];
        annotations_tmp["ignore"] = 0;
        annotations_tmp["area"] = (x2-x1)*(y2-y1);
        annotations_tmp["iscrowd"] = 0;
        annotations_tmp["bbox"] = [x1, y1, x2-x1, y2-y1];
        annotations_tmp["category_id"] = anno.index(label);

        coco_data["annotations"].append(annotations_tmp)
    image_id += 1;

outfile =  open(output_annotation_file, 'w');
json_str = json.dumps(coco_data, indent=4);
outfile.write(json_str);
outfile.close();

# Training

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/16_mmdet/lib")

In [ ]:
from train_engine import Detector

In [ ]:
gtf = Detector();

In [ ]:
img_dir = "dataset/train/images";
annofile = "dataset/train/annotations/instances_images.json"
class_file = "dataset/train/annotations/classes.txt"

gtf.Train_Dataset(img_dir, annofile, class_file);

In [ ]:
img_dir = "dataset/valid/images";
annofile = "dataset/valid/annotations/instances_images.json"

gtf.Val_Dataset(img_dir, annofile);

In [ ]:
gtf.Dataset_Params(batch_size=2, num_workers=4)

In [ ]:
gtf.List_Models();

In [ ]:
gtf.Model_Params(model_name="cascade_rcnn_fpn101");

In [ ]:
gtf.Hyper_Params(lr=0.02, momentum=0.9, weight_decay=0.0001);

In [ ]:
gtf.Training_Params(num_epochs=10, val_interval=3);

In [ ]:
gtf.Train();

# Run inference on images

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/16_mmdet/lib")

In [ ]:
from infer_engine import Infer

In [ ]:
gtf = Infer();

In [ ]:
gtf.Model_Params("work_dirs/config_updated/config_updated.py", 
                 "work_dirs/config_updated/latest.pth")

In [ ]:
import os
img_list = os.listdir("dataset/test/images");

In [ ]:
result = gtf.Predict(img_path="dataset/test/images/" + img_list[0],
           out_img_path="result.jpg",
           thresh=0.3);

from IPython.display import Image
Image(filename='result.jpg', width=490, height=640) 

In [ ]:
result = gtf.Predict(img_path="dataset/test/images/" + img_list[2],
           out_img_path="result.jpg",
           thresh=0.3);

from IPython.display import Image
Image(filename='result.jpg', width=490, height=640) 